In [1]:
import sys
import datetime as dt
from pathlib import Path
from collections import OrderedDict
import math
from shutil import copyfile

import requests
import gtfstk as gt
from highcharts import Highchart 

DIR = Path('..')
sys.path.append(str(DIR))

%load_ext autoreload
%autoreload 2

from gtfsrtk import *

TESTS_DIR = DIR/'tests'/'data'
DATA_DIR = DIR/'data'

In [ ]:
# Build a function to get GTFSr trip update feeds
api_key = ut.get_secret('AUCKLAND_API_KEY')
url = 'https://api.at.govt.nz/v2/public/realtime/?'#tripupdates?'
headers = {
  'Ocp-Apim-Subscription-Key': api_key,
  }
params = {
  'callback': '',
  'tripid': '',
  'vehicleid': '',
  }    

get_feed = build_get_feed(url, headers, params)
feed = get_feed()
feed['response']['header']


In [ ]:
extract_delays(feed)[0]

In [2]:
# Create augmented stop times

gtfsr_dir = DATA_DIR/'auckland_gtfsr_trip_updates'
feed = gt.read_gtfs(DATA_DIR/'auckland_gtfs_20160519.zip', dist_units_in='km') # Good from 20160519
date = '20160519'
ast = build_augmented_stop_times(gtfsr_dir, feed, date)
ast.head(30)

Timing create_augmented_stop_times...
2016-07-06 15:25:36.369286   Began process
2016-07-06 15:27:28.448422   Finished in 1.87 min


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_delay,departure_delay
0,1000015308-20160512154122_v40.34,07:00:00,07:00:00,6920,1,NaN,NaN,NaN,0.000000,NaN,NaN
1,1000015308-20160512154122_v40.34,07:02:10,07:02:10,6800,2,NaN,NaN,NaN,1.184603,NaN,108.0
2,1000015308-20160512154122_v40.34,07:02:20,07:02:20,6364,3,NaN,NaN,NaN,1.474505,NaN,NaN
3,1000015308-20160512154122_v40.34,07:02:30,07:02:30,6366,4,NaN,NaN,NaN,1.740877,NaN,NaN
4,1000015308-20160512154122_v40.34,07:02:40,07:02:40,6182,5,NaN,NaN,NaN,2.013814,NaN,151.0
5,1000015308-20160512154122_v40.34,07:03:03,07:03:03,6156,6,NaN,NaN,NaN,2.289616,200.0,208.0
6,1000015308-20160512154122_v40.34,07:03:28,07:03:28,6319,7,NaN,NaN,NaN,2.886459,NaN,NaN
7,1000015308-20160512154122_v40.34,07:03:38,07:03:38,6306,8,NaN,NaN,NaN,3.137981,NaN,337.0
8,1000015308-20160512154122_v40.34,07:03:46,07:03:46,6308,9,NaN,NaN,NaN,3.452160,NaN,NaN
9,1000015308-20160512154122_v40.34,07:04:02,07:04:02,6313,10,NaN,NaN,NaN,3.650870,376.0,398.0


In [3]:
# Clean augmented stop times

f = clean_augmented_stop_times(ast, dist_threshold=0.5)
f.head(30)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,delay
0,1000015308-20160512154122_v40.34,07:00:00,07:00:00,6920,1,NaN,NaN,NaN,0.000000,0.0
1,1000015308-20160512154122_v40.34,07:02:10,07:02:10,6800,2,NaN,NaN,NaN,1.184603,108.0
2,1000015308-20160512154122_v40.34,07:02:20,07:02:20,6364,3,NaN,NaN,NaN,1.474505,123.0
3,1000015308-20160512154122_v40.34,07:02:30,07:02:30,6366,4,NaN,NaN,NaN,1.740877,137.0
4,1000015308-20160512154122_v40.34,07:02:40,07:02:40,6182,5,NaN,NaN,NaN,2.013814,151.0
5,1000015308-20160512154122_v40.34,07:03:03,07:03:03,6156,6,NaN,NaN,NaN,2.289616,208.0
6,1000015308-20160512154122_v40.34,07:03:28,07:03:28,6319,7,NaN,NaN,NaN,2.886459,299.0
7,1000015308-20160512154122_v40.34,07:03:38,07:03:38,6306,8,NaN,NaN,NaN,3.137981,337.0
8,1000015308-20160512154122_v40.34,07:03:46,07:03:46,6308,9,NaN,NaN,NaN,3.452160,374.0
9,1000015308-20160512154122_v40.34,07:04:02,07:04:02,6313,10,NaN,NaN,NaN,3.650870,398.0


In [ ]:
# Compute trips stats for later
ts = gt.compute_trips_stats(feed)
# rs = gt.compute_routes_stats(feed, ts, date)
# rs = rs.merge(feed.routes[['route_id', 'route_long_name']])


# Tukey box plots of stop delays by route

In [ ]:
def agg_tukey(group):
    s = dict()
    dcol = 'delay'
    for i in range(1, 4):
        col = 'delay_q{0}'.format(i)
        s[col] = group[dcol].dropna().quantile(i/4)
          
    iqr = s['delay_q3'] - s['delay_q1']
    s['iqr'] = iqr
    cond = group[dcol] <= s['delay_q3'] + 1.5*iqr
    s['delay_high'] = group.loc[cond, dcol].max()
    cond = group[dcol] >= s['delay_q1'] - 1.5*iqr
    s['delay_low'] = group.loc[cond, dcol].min()
    s['num_delays'] = group[dcol].count()
    s['num_realtime_runs'] = (group['trip_id'] + group['date']).nunique()
    s['num_unique_trip_ids'] = group['trip_id'].nunique()
    return pd.Series(s)

f = all_delays.copy()
f = f.groupby('route_id').apply(agg_tukey).reset_index()
f = f.merge(feed.routes)
f.head()

# Only keep routes with an average of at least k delay samples per sample trip
k = 3
cond = f['num_delays']/f['num_realtime_runs'] >= k
g = f[cond].copy()

# Convert to minutes
box_cols = ['delay_low', 'delay_q1', 'delay_q2', 'delay_q3', 'delay_high']
g[box_cols] /= 60
g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
  lambda x: ', '.join(x), axis=1) 

# Print 10 biggest IQR routes
print('Routes with largest IQR:')
print(g.sort_values('iqr', ascending=False).head(5))

# Plot
chart = Highchart()
data = g[['route_name'] + box_cols].copy()
chart.add_data_set(data.values.tolist(), series_type='boxplot')
N = data.shape[0]
date_range = ', '.join(dates)        
    
options = {
    'chart': {
        'height': 15*N,
        'inverted': True,
    },
    'plotOptions': {
    },
    'title': {
        'text': 'Tukey box plots of stop delays by route',
    },
    'subtitle': {
        'text': 'Sample period is {0}'.format(date_range)
    },
    'legend': {
        'enabled': False,
    },
    'xAxis': {
        'title': {
            'text': 'Route'
        },
        'type': 'category',
        'labels': {'step': 1},
    },
    'yAxis': {
        'title': {
            'text': 'Minutes'
        },
        'opposite': True,
        'plotLines': [{
            'value': 0,
            'color': 'red',
            'width': 2,
        }]    
    },
    'tooltip': {
#         'formatter': 'function () {\
#             return this.point.q3 - this.point.q1 + " min"\
#         }',
        'valueDecimals': 1,
        'headerFormat': '<b>{point.key}</b><br/>',
        'pointFormat': 'low: {point.low} min<br/>'\
          'Q1: {point.q1} min<br/>'\
          'Q2: {point.median} min<br/>'\
          'Q3: {point.q3} min<br/>'\
          'high: {point.high} min',
    }
}
chart.set_dict_options(options)
chart

In [ ]:
# Select a time-of-day window
t1 = '07:00:00'
t2 = '09:00:00'

cols = ['trip_id', 'start_time', 'direction_id']
f = all_delays.copy().merge(ts[cols])

time_cond = (f['start_time'] >= t1) & (f['start_time'] <= t2)

# Separate directions
charts = []
for direction in [0, 1]:
    dir_cond = f['direction_id'] == direction
    cond = time_cond & dir_cond
    g = f.loc[cond].copy()
    g = g.groupby('route_id').apply(agg_for_box_plot).reset_index()
    g = g.merge(feed.routes)
    g.head()

    # Only keep routes with an average of at least k delay samples per sample trip
    k = 3
    cond = g['num_delays']/g['num_realtime_runs'] >= k
    g = g[cond].copy()

    # Convert to minutes
    box_cols = ['delay_low', 'delay_q1', 'delay_q2', 'delay_q3', 'delay_high']
    g[box_cols] /= 60
    g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
      lambda x: ', '.join(x), axis=1) 

    # Print 10 biggest IQR routes
    print('Routes with largest IQR:')
    print(g.sort_values('iqr', ascending=False).head(5))

    # Plot
    chart = Highchart()
    data = g[['route_name'] + box_cols].copy()
    chart.add_data_set(data.values.tolist(), series_type='boxplot')
    N = data.shape[0]
    date_range = ', '.join(dates)        

    options = {
        'chart': {
            'height': 15*N,
            'inverted': True,
        },
        'plotOptions': {
        },
        'title': {
            'text': 'Tukey box plots of stop delays by route in direction {0}'.format(direction),
        },
        'subtitle': {
            'text': 'Time of day is [{0}, {1}]. <br/>Sample period is {2}'.format(
            t1, t2, date_range)
        },
        'legend': {
            'enabled': False,
        },
        'xAxis': {
            'title': {
                'text': 'Route'
            },
            'type': 'category',
            'labels': {'step': 1},
        },
        'yAxis': {
            'title': {
                'text': 'Minutes'
            },
            'opposite': True,
            'plotLines': [{
                'value': 0,
                'color': 'red',
                'width': 2,
            }]    
        },
        'tooltip': {
    #         'formatter': 'function () {\
    #             return this.point.q3 - this.point.q1 + " min"\
    #         }',
            'valueDecimals': 1,
            'headerFormat': '<b>{point.key}</b><br/>',
            'pointFormat': 'low: {point.low} min<br/>'\
              'Q1: {point.q1} min<br/>'\
              'Q2: {point.median} min<br/>'\
              'Q3: {point.q3} min<br/>'\
              'high: {point.high} min',
        }
    }
    chart.set_dict_options(options)
    charts.append(chart)
    

In [ ]:
charts[0]

In [ ]:
charts[1]

# Tukey box plots of end-stop delays by route

In [ ]:
f = all_delays.copy()

# Compute trip end stop sequences
st = feed.stop_times.sort_values(['trip_id', 'stop_sequence'])
st = st.groupby('trip_id').agg('last')['stop_sequence'].reset_index()

# Get delays only for end stops
f = f.merge(st)

f.head(20)

# Aggregate by route
f = f.groupby('route_id').apply(agg_tukey).reset_index()
f = f.merge(feed.routes)

# Only keep routes with an average of at least k end-stop delays per unique trip ID
k = 2
cond = f['num_delays']/f['num_unique_trip_ids'] >= k
g = f[cond].copy()

# Convert to minutes
box_cols = ['delay_low', 'delay_q1', 'delay_q2', 'delay_q3', 'delay_high']
g[box_cols] /= 60
g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
  lambda x: ', '.join(x), axis=1) 

# Print biggest IQR routes
print('Routes with largest IQR:')
print(g.sort_values('iqr', ascending=False).head(5))

# Plot
chart = Highchart()
data = g[['route_name'] + box_cols].copy()
chart.add_data_set(data.values.tolist(), series_type='boxplot')
N = data.shape[0]
date_range = ', '.join(dates)        
    
options = {
    'chart': {
        'height': 15*N,
        'inverted': True,
    },
    'plotOptions': {
    },
    'title': {
        'text': 'Tukey box plots of end-stop delays by route',
    },
    'subtitle': {
        'text': 'Sample period is {0}'.format(date_range)
    },
    'legend': {
        'enabled': False,
    },
    'xAxis': {
        'title': {
            'text': 'Route'
        },
        'type': 'category',
        'labels': {'step': 1},
    },
    'yAxis': {
        'title': {
            'text': 'Minutes'
        },
        'opposite': True,
        'plotLines': [{
            'value': 0,
            'color': 'red',
            'width': 2,
        }]    
    },
    'tooltip': {
#         'formatter': 'function () {\
#             return this.point.q3 - this.point.q1 + " min"\
#         }',
        'valueDecimals': 1,
        'headerFormat': '<b>{point.key}</b><br/>',
        'pointFormat': 'low: {point.low} min<br/>'\
          'Q1: {point.q1} min<br/>'\
          'Q2: {point.median} min<br/>'\
          'Q3: {point.q3} min<br/>'\
          'high: {point.high} min',
    }
}
chart.set_dict_options(options)
chart

# Explore percentage of punctual trips by route

In [ ]:
# Say that a trip is k-punctual if it arrives no more than
# k seconds late at its final stop

k = 5*60

# Get trip final delays
def final_delay(group):
    d = dict()
    d['final_delay'] = group['arrival_delay'].iat[-1]
    return pd.Series(d)

f = st.copy()
f = f.sort_values(['trip_id', 'stop_sequence']).groupby('trip_id').apply(
  final_delay).reset_index()
f = f.merge(ts)

# Calculate fraction of k-punctual trips for each route
def fpt(group):
    d = dict()
    n = group['final_delay'].count()
    cond = group['final_delay'] <= k
    d['frac_punctual_trips'] = group.loc[cond, 'final_delay'].shape[0]/n
    d['num_samples'] = n
    return pd.Series(d)

f = f.groupby('route_id').apply(fpt).reset_index()
f = f.merge(rs)
f['frac_samples'] = f['num_samples']/f['num_trips']
f.sort_values('frac_punctual_trips').head(10)

In [ ]:
# Plot some

# Only keep routes with at least half of trips sampled
cond = f['frac_samples'] >= 0.5
g = f[cond].copy()
g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
  lambda x: ', '.join(x), axis=1) 
g['ppt'] = g['frac_punctual_trips']*100

chart = Highchart()
data = g[['route_name', 'ppt']].copy()
chart.add_data_set(data.values.tolist(), series_type='bar')
N = data.shape[0]
        
options = {
    'chart': {
        'height': 15*N,
    },
    'plotOptions': {
        'series': {
            'minPointLength': 3,
        },
    },
    'title': {
        'text': 'Percentage of 5-minute-punctual trips by route',
    },
    'subtitle': {
        'text': 'Sample period is {0}'.format(date)
    },
    'legend': {
        'enabled': False,
    },
    'xAxis': {
        'title': {
            'text': 'Route'
        },
        'type': 'category',
        'labels': {'step': 1},
    },
    'yAxis': {
        'title': {
            'text': 'Percentage'
        },
        'opposite': True,
        'max': 100,
    },
    'tooltip': {
        'headerFormat': '<b>{point.key}</b><br/>',
        'pointFormat': '{point.y}%',
        'valueDecimals': 0,
    }
}
chart.set_dict_options(options)
chart

In [ ]:
# # Fill departure delays.

# # 1. First pass. 
# # Fill actual departure times via distance interpolation.
# # Then use these to fill departure delays.
# f = st.copy()

# print('num trips=', f['trip_id'].nunique())

# f['actual_departure_time'] = f['departure_time'].map(gt.timestr_to_seconds) +\
#   f['departure_delay']

# def fill_dtimes(group):
#     indices = np.where(group['actual_departure_time'].notnull())[0]
#     if not indices.size:
#         return group
#     times = group['actual_departure_time'].ix[indices]
#     dists = group['shape_dist_traveled'].ix[indices]
#     ds = group['shape_dist_traveled']
#     ts = np.interp(ds, dists, times)
#     group['actual_departure_time'] = ts
#     return group

# f = f.groupby('trip_id').apply(fill_dtimes)

# # Round departure times
# f['actual_departure_time'] = f['actual_departure_time'].round()

# # Compute delays
# f['departure_delay'] = f['actual_departure_time'] -\
#   f['arrival_time'].map(gt.timestr_to_seconds)
    

# # 2. Second pass. 
# # For the remaining null departure delays, fill them by forward fill
# # and then backward fill.
# f['departure_delay'] = f['departure_delay'].fillna(method='ffill').fillna(method='bfill')     

# f['actual_departure_time'] = f['departure_time'].map(gt.timestr_to_seconds) +\
#   f['departure_delay']

# # Convert back to time strings
# f['actual_departure_time'] = f['actual_departure_time'].map(
#   lambda x: gt.timestr_to_seconds(x, inverse=True))

# f.T